In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##  Урок 12. Модель Transformer-2

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует

In [10]:
import json
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import pandas as pd
import numpy as np
from datasets import load_metric

In [4]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

No config specified, defaulting to: gazeta/default
Reusing dataset gazeta (/root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame(dataset)[:100]

In [8]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [9]:
predictions = []

for i in df['text']:
    input_ids = tokenizer(
        [i],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    prediction = tokenizer.decode(output_ids, skip_special_tokens=True)
    predictions.append(prediction)

In [15]:
rouge_score = load_metric("rouge")

sum_scores = rouge_score.compute(
    predictions=[predictions], references=[df['summary']]
)
sum_scores

{'rouge1': AggregateScore(low=Score(precision=0.527027027027027, recall=0.6724137931034483, fmeasure=0.5909090909090909), mid=Score(precision=0.527027027027027, recall=0.6724137931034483, fmeasure=0.5909090909090909), high=Score(precision=0.527027027027027, recall=0.6724137931034483, fmeasure=0.5909090909090909)),
 'rouge2': AggregateScore(low=Score(precision=0.20408163265306123, recall=0.2608695652173913, fmeasure=0.22900763358778625), mid=Score(precision=0.20408163265306123, recall=0.2608695652173913, fmeasure=0.22900763358778625), high=Score(precision=0.20408163265306123, recall=0.2608695652173913, fmeasure=0.22900763358778625)),
 'rougeL': AggregateScore(low=Score(precision=0.39864864864864863, recall=0.5086206896551724, fmeasure=0.4469696969696969), mid=Score(precision=0.39864864864864863, recall=0.5086206896551724, fmeasure=0.4469696969696969), high=Score(precision=0.39864864864864863, recall=0.5086206896551724, fmeasure=0.4469696969696969)),
 'rougeLsum': AggregateScore(low=Scor